In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
# UPLOAD TRAIN AND TEST
import pandas as pd
import numpy as np
train = pd.read_csv("/content/drive/My Drive/Thesis/Git_repo/Data/train.csv")
train_val = pd.read_csv("/content/drive/My Drive/Thesis/Git_repo/Data/train_val.csv")
val = pd.read_csv("/content/drive/My Drive/Thesis/Git_repo/Data/validation.csv")
test = pd.read_csv("/content/drive/My Drive/Thesis/Git_repo/Data/test.csv") # equal for all the models
print("Train shape ", train.shape)
print("Validation shape ", val.shape)
print("Test shape ", test.shape)

Train shape  (640303, 3)
Validation shape  (159799, 3)
Test shape  (199898, 3)


## BASELINES

In [6]:
pip install surprise

     |████████████████████████████████| 6.5MB 6.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675356 sha256=3250ed510ce39e0cceccc6fda5a1cc4dfabc287368070908d514951288ebd771
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
import surprise
from surprise import Reader
import os
os.chdir('/content/drive/My Drive/Thesis/Git_repo') # select position of repo
from utils_surprise.surprise_functions import fit_predict,predict,compute_rmse,epochs_tuning

In [0]:
# import data in surprise format
reader = Reader(rating_scale=(1, 5))
surprise_train = surprise.Dataset.load_from_df(train, reader=reader).build_full_trainset()

In [0]:
n_epochs = [10,20,30,40,50,60,70,80] # epochs to tune
rmse_epochs = epochs_tuning(surprise_train,val,n_epochs, PMF = False) # n_factors is fixed to 10

In [7]:
rmse_epochs

[(0.9976730766304444, 10),
 (0.9904701620957754, 20),
 (0.9926401566411966, 30),
 (1.0008064165239408, 40),
 (1.011456538934997, 50),
 (1.0219140196350176, 60),
 (1.031099231494102, 70),
 (1.0389012572593574, 80)]

In [0]:
np.save("/content/drive/My Drive/Thesis/Git_repo/Data/SVD_tuning_epochs",rmse_epochs)

In [0]:
# run with best n_epochs found with the previous function and n_factors = 10
result , model = fit_predict(surprise_train,val,surprise.SVD,n_factors=10,n_epochs=20,use_PMF = False)

In [0]:
# prediction with the microsoft function
SVD_pred = predict(model, test, usercol='userID', itemcol='movieID',predcol="rating")

In [0]:
np.save("/content/drive/My Drive/Thesis/Git_repo/Data/SVD_pred",SVD_pred["rating"].values)

In [0]:
# compute rmse baselines
true_pred = test["rating"].values
pred = SVD_pred["rating"].values
rmse = compute_rmse(true_pred,pred)

In [0]:
rmse

0.9880064434911993

## FASTAI

In [0]:
import fastai
print(fastai.__version__)
from fastai import *
from fastai.collab import *
from fastai.callbacks import *
import math

os.chdir('/content/drive/My Drive/Thesis/Git_repo') # select position of repo
from utils_fastai.fastai_functions import MLP_fastai_find_best_layers,MF_fastai_Gsearch

1.0.61


In [0]:
# test data for fastai
test_fastai = test[["userID","movieID"]]

### MF

In [0]:
n_batch_sizes = [128,256,512,1024]
lr = [0.0001 ,0.0005, 0.001, 0.005]

In [0]:
df = MF_fastai_Gsearch(train_val,n_batch_sizes,lr,nr_epochs=20) # early stopping is implemented to find best n_epochs. n_factor is fixed to 10

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,1.532903,1.536684,1.238441,01:17
1,1.531003,1.519567,1.231520,01:17
2,1.489368,1.484746,1.217321,01:17
3,1.420330,1.433757,1.196218,01:16
4,1.379623,1.375145,1.171484,01:17
5,1.307325,1.319501,1.147498,01:18
6,1.270254,1.273470,1.127259,01:19
7,1.241828,1.237068,1.110984,01:18
8,1.212026,1.208874,1.098206,01:16
9,1.145741,1.187272,1.088308,01:17


Epoch 17: early stopping


epoch,train_loss,valid_loss,root_mean_squared_error,time
0,1.530073,1.541087,1.240805,00:59
1,1.547819,1.528675,1.235797,00:59
2,1.498964,1.502994,1.225371,01:00
3,1.463733,1.464117,1.209415,01:00
4,1.406744,1.417305,1.189916,00:59
5,1.348667,1.369982,1.169873,00:59
6,1.309904,1.328021,1.151804,00:59
7,1.265790,1.292498,1.136281,00:58
8,1.242263,1.263165,1.123299,00:59
9,1.206606,1.239400,1.112667,00:59


Epoch 18: early stopping


epoch,train_loss,valid_loss,root_mean_squared_error,time
0,1.550122,1.542527,1.241703,00:50
1,1.535341,1.533843,1.238203,00:50
2,1.504975,1.515641,1.230834,00:50
3,1.492986,1.487458,1.219336,00:51
4,1.457559,1.452296,1.204835,00:50
5,1.410732,1.415171,1.189334,00:51
6,1.368896,1.380605,1.174716,00:51
7,1.331604,1.349863,1.161559,00:51
8,1.305543,1.323119,1.149991,00:51
9,1.277975,1.300418,1.140078,00:52


...

Epoch 6: early stopping


epoch,train_loss,valid_loss,root_mean_squared_error,time
0,1.426294,1.416119,1.189732,00:55
1,1.189066,1.193654,1.092243,00:55
2,0.983980,1.065421,1.031815,00:55
3,0.739901,1.068365,1.033202,00:55
4,0.516308,1.129000,1.062074,00:55
5,0.368886,1.196544,1.093356,00:54


Epoch 6: early stopping


epoch,train_loss,valid_loss,root_mean_squared_error,time
0,1.461765,1.455424,1.206276,00:53
1,1.275785,1.264594,1.124414,00:53
2,1.034896,1.100014,1.048668,00:54
3,0.772681,1.066823,1.032693,00:53
4,0.525777,1.118644,1.057447,00:53
5,0.353175,1.185249,1.088437,00:54
6,0.256307,1.235405,1.111212,00:54


Epoch 7: early stopping


In [0]:
df

,128,256,512,1024
Learning_rate,,,,
0.0001,1.064081,1.082681,1.108868,1.140043
0.0005,1.038386,1.032578,1.036230,1.037506
0.0010,1.053499,1.047563,1.047444,1.046622
0.0050,1.098876,1.108339,1.111744,1.131219


In [0]:
df.to_csv("/content/drive/My Drive/Thesis/Git_repo/Data/MF_Gsearch.csv",index=True)

In [0]:
# training best MF model
data = CollabDataBunch.from_df(train_val, valid_pct=0.2, test=test_fastai,seed=2,bs=256)
learn = collab_learner(data, n_factors=10, y_range=(1,5) , metrics=[rmse],
                           callback_fns=[partial(EarlyStoppingCallback, min_delta=0.01, patience=2, mode="min", monitor="root_mean_squared_error")])
learn.fit_one_cycle(20, 0.0005)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,1.519139,1.526347,1.234856,01:03
1,1.473074,1.468949,1.211410,01:03
2,1.362170,1.367030,1.168611,01:04
3,1.236591,1.252438,1.118514,01:04
4,1.131311,1.164297,1.078368,01:03
5,1.035039,1.112307,1.053948,01:04
6,0.948816,1.084947,1.040853,01:05
7,0.890267,1.071187,1.034200,01:04
8,0.798510,1.065482,1.031423,01:04


Epoch 9: early stopping


In [0]:
# prediction best MF model
MF_pred = learn.get_preds(DatasetType.Test)

In [0]:
np.save("/content/drive/My Drive/Thesis/Git_repo/Data/MF_pred",MF_pred[0])

In [0]:
# RMSE on test set best MF model
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(test["rating"].values,MF_pred[0]))

1.0296562471666209

### MLP fastai

In [0]:
# find best set of layers
layers = [[256,128,64], #best
          [128,64,32],
          [64,32,16], 
          [32,16,8]]

# it returns a tuple with min rmse and correspondent layers
result = MLP_fastai_find_best_layers(train_val,n_batch_sizes = 256 ,lr = 0.0005, nr_epochs = 2 ,layers = layers) 

epoch,train_loss,valid_loss,time
0,1.035990,1.037878,01:13
1,0.918822,1.019447,01:14


epoch,train_loss,valid_loss,time
0,1.055989,1.036942,01:14
1,0.911771,1.026414,01:14


epoch,train_loss,valid_loss,time
0,1.038432,1.041409,01:13
1,0.923832,1.026529,01:12


epoch,train_loss,valid_loss,time
0,1.031700,1.037814,01:13
1,0.897006,1.033982,01:14


In [0]:
result

(1.0096768426878753, [256, 128, 64])

In [0]:
# training MLP with the best set of layers
data = CollabDataBunch.from_df(train_val, valid_pct=0.2, test=test_fastai,seed=2,bs=256)
learn_MLP = collab_learner(data, use_nn=True, emb_szs={'userID': 10, 'movieID':10}, layers= [256,128,64], y_range=[1, 5])
learn_MLP.fit_one_cycle(2,0.0005) 

epoch,train_loss,valid_loss,time
0,1.021285,1.037750,01:16
1,0.926001,1.017189,01:17


In [0]:
# val RSME
rmse = learn_MLP.validate(learn.data.valid_dl)
math.sqrt(rmse[0])

1.0085578350643143

In [0]:
# prediction best MLP model
from sklearn.metrics import mean_squared_error
from math import sqrt
pred_MLP = learn_MLP.get_preds(DatasetType.Test)
sqrt(mean_squared_error(test["rating"].values,pred_MLP[0]))

1.0047666989174364

In [0]:
np.save("/content/drive/My Drive/Thesis/Git_repo/Data/MLP_pred",pred_MLP[0])

## NMF

In [0]:
import os
os.chdir('/content/drive/My Drive/Thesis/Git_repo') # select position of repo
from Models.nmf_models import MF_layer, MLP_layer, NMF, NMF_Gsearch,NMF_find_best_layers
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from math import sqrt

In [0]:
n_batch_sizes = [128,256,512,1024]
lr = [0.0001 ,0.0005, 0.001, 0.005]

# it runs NFM with MLP layers = [30,20,10] and NMF layer = 30. n_factors is always 10
# it returns a df with index = lr and columns = batch sizes. values = rmse
# models run with early stopping
df = NMF_Gsearch(train = train,val = val ,n_batch_sizes = n_batch_sizes,lr = lr ,n_epochs = 30)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
df # batch size and lr do not affect rmse

,128,256,512,1024
Learning_rate,,,,
0.0001,1.030346,1.030616,1.031371,1.034364
0.0005,1.030425,1.030016,1.030339,1.030257
0.0010,1.031062,1.031265,1.031937,1.030512
0.0050,1.032233,1.035004,1.032986,1.031324


In [0]:
layers = [[256,128,64], 
          [128,64,32],
          [64,32,16],
          [32,16,8]]
result = NMF_find_best_layers(train = train ,val = val ,n_batch_sizes = 256 ,lr = 0.0001,nr_epochs = 30 ,layers = layers)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
result

(1.0303803076945743, [256, 128, 64])

In [0]:
# train best NMF with best lr batch size and layers
model = NMF(n_movies=len(train['movieID'].unique()),n_users=len(train['userID'].unique()),
           n_latent_factors=10,MLP_layers=[256,128,64],NMF_layer_units = 256, lr=0.0001)

In [0]:
# set hyperparameters 
batch_size=256
epochs=30
es = EarlyStopping(monitor = "val_loss", patience = 3, mode = "min", verbose = 1)

In [0]:
History = model.fit([train.userID,train.movieID],train.rating, batch_size=batch_size,
                              epochs = epochs, validation_data = ([val.userID,val.movieID],val.rating),callbacks=[es],
                              verbose = 1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 640303 samples, validate on 159799 samples
Epoch 1/30
640303/640303 [==============================] - 31s 49us/step - loss: 1.6373 - val_loss: 1.0760
Epoch 2/30
640303/640303 [==============================] - 29s 46us/step - loss: 1.0577 - val_loss: 1.0636
Epoch 3/30
640303/640303 [==============================] - 30s 46us/step - loss: 1.0362 - val_loss: 1.0723
Epoch 4/30
640303/640303 [==============================] - 30s 46us/step - loss: 1.0209 - val_loss: 1.0648
Epoch 5/30
640303/640303 [==============================] - 30s 47us/step - loss: 1.0095 - val_loss: 1.0672
Epoch 00005: early stopping


In [0]:
# rmse val
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(History.history['val_loss']))
print ('Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(sqrt(min_val_loss)))

Minimum RMSE at epoch 2 = 1.0304


In [0]:
# predict on test
prediction = model.predict([test.userID,test.movieID])

In [0]:
# rmse on test
sqrt(mean_squared_error(test["rating"].values,prediction))

1.0310921197728746

In [0]:
np.save("/content/drive/My Drive/Thesis/Git_repo/Data/NMF_pred",prediction)

## CONFUSION MATRIX

In [0]:
MF_pred = np.load("/content/drive/My Drive/Thesis/Git_repo/Data/MF_pred.npy")
MLP_pred = np.load("/content/drive/My Drive/Thesis/Git_repo/Data/MLP_pred.npy")
NMF_pred = np.load("/content/drive/My Drive/Thesis/Git_repo/Data/NMF_pred.npy")
SVD_pred = np.load("/content/drive/My Drive/Thesis/Git_repo/Data/SVD_pred.npy")
test = pd.read_csv("/content/drive/My Drive/Thesis/Git_repo/Data/test.csv") # equal for all the models
true_pred = test[["rating"]].values
MF_pred = MF_pred.astype('int64')
MLP_pred = MLP_pred.astype('int64')
NMF_pred = NMF_pred.astype('int64')
SVD_pred = SVD_pred.astype("int64")
classes = [1,2,3,4,5]

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

In [0]:
MF_accuracy = accuracy_score(true_pred, MF_pred)
print('Accuracy: %f' % MF_accuracy)
MF_scores = precision_recall_fscore_support(true_pred, MF_pred,average='weighted')
print(MF_scores)

Accuracy: 0.289363
(0.25209763601720664, 0.2893625749132057, 0.18569105059468347, None)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
MLP_accuracy = accuracy_score(true_pred, MLP_pred)
print('Accuracy: %f' % MLP_accuracy)
MLP_scores = precision_recall_fscore_support(true_pred, MLP_pred,average='weighted')
print(MLP_scores)

Accuracy: 0.317017
(0.2612109414474344, 0.3170166785060381, 0.25320592821396126, None)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
NMF_accuracy = accuracy_score(true_pred, NMF_pred)
print('Accuracy: %f' % NMF_accuracy)
NMF_scores = precision_recall_fscore_support(true_pred, NMF_pred,average='weighted')
print(NMF_scores)

Accuracy: 0.302234
(0.4038493357606905, 0.30223413941109967, 0.21303185681392325, None)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
SVD_accuracy = accuracy_score(true_pred, SVD_pred)
print('Accuracy: %f' % SVD_accuracy)
SVD_scores = precision_recall_fscore_support(true_pred, SVD_pred,average='weighted')
print(SVD_scores)

Accuracy: 0.316862
(0.46031200169616493, 0.316861599415702, 0.23649501261181688, None)


In [0]:
confusion_matrix(true_pred , MLP_pred, labels = classes)

array([[  257,  2443,  5495,   941,     0],
       [  101,  4909, 13155,  1876,     0],
       [   54,  8760, 40618,  7782,     0],
       [   21,  4722, 45226, 17587,     0],
       [   12,  1496, 24043, 20400,     0]])

In [0]:
confusion_matrix(true_pred,SVD_pred,labels = classes)

array([[  226,  2132,  6156,   622,     0],
       [   92,  3756, 15088,  1104,     1],
       [   35,  5563, 46679,  4937,     0],
       [   18,  2553, 52360, 12617,     8],
       [   14,   783, 27982, 17110,    62]])

In [0]:
SVD_pred

array([2, 3, 3, 3, ..., 2, 4, 3, 3])